<a href="https://colab.research.google.com/github/patrycjapiechowicz/hobby-projects/blob/main/Regression_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression problem - diamond prices prediction


In [ ]:
pip install rapids

In [ ]:
!pip install cuml-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!pip install cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!rm -rf /usr/local/lib/python3.8/dist-packages/cupy*
!pip install cupy-cuda11x

In [ ]:
import numpy as np        
import pandas as pd       
import matplotlib.pyplot as plt
import seaborn as sns 

import cudf
import cupy as cp
import cuml


### Data import and preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = cudf.read_csv("/content/diamonds.csv")

In [ ]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  53940 non-null  object
 1   carat       53940 non-null  float64
 2   cut         53940 non-null  object
 3   color       53940 non-null  object
 4   clarity     53940 non-null  object
 5   depth       53940 non-null  float64
 6   table       53940 non-null  float64
 7   price       53940 non-null  int64
 8   x           53940 non-null  float64
 9   y           53940 non-null  float64
 10  z           53940 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 4.5+ MB


In [ ]:
df.drop(columns = "Unnamed: 0",inplace=True)

In [ ]:
# train/test split

from sklearn.model_selection import train_test_split as sk_train_test_split
from cuml.model_selection import train_test_split

X = df.drop(columns = "price")
y = df['price']

In [ ]:
X.memory_usage()

carat      431520
cut        554858
color      269704
clarity    383774
depth      431520
table      431520
x          431520
y          431520
z          431520
Index           0
dtype: int64

In [ ]:
# cast na typ category i podanie do xgboost takich danych - istotna redukcja utylizacji pamieci
X['color'] = X['color'].astype("category")
X['cut'] = X['cut'].astype("category")
X['clarity'] = X['clarity'].astype("category")


In [ ]:
df.cut.value_counts()

Ideal        21551
Premium      13791
Very Good    12082
Good          4906
Fair          1610
Name: cut, dtype: int32

In [ ]:
X.memory_usage()

carat      431520
cut         53993
color       53979
clarity     54000
depth      431520
table      431520
x          431520
y          431520
z          431520
Index           0
dtype: int64

In [ ]:
X_train, X_test,y_train,y_test = \
train_test_split(X,y,test_size=.3,random_state=42)

In [ ]:
?sk_train_test_split

In [ ]:
X_train.info()

<class 'cudf.core.dataframe.DataFrame'>
Int64Index: 37758 entries, 45886 to 2363
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   carat    37758 non-null  float64
 1   cut      37758 non-null  object
 2   color    37758 non-null  object
 3   clarity  37758 non-null  object
 4   depth    37758 non-null  float64
 5   table    37758 non-null  float64
 6   x        37758 non-null  float64
 7   y        37758 non-null  float64
 8   z        37758 non-null  float64
dtypes: float64(6), object(3)
memory usage: 2.8+ MB


In [ ]:
df_p = X.to_pandas()

In [ ]:
df_p['color'].astype("category").factorize()

(array([0, 0, 0, ..., 6, 3, 6]),
 CategoricalIndex(['E', 'I', 'J', 'H', 'F', 'G', 'D'], categories=['D', 'E', 'F', 'G', 'H', 'I', 'J'], ordered=False, dtype='category'))

In [ ]:
?pd.Series.factorize

In [ ]:
from cuml.preprocessing import StandardScaler, FunctionTransformer,OneHotEncoder
from cuml.compose import ColumnTransformer
from cuml.pipeline import Pipeline

In [ ]:
df['cut'].value_counts()

Ideal        21551
Premium      13791
Very Good    12082
Good          4906
Fair          1610
Name: cut, dtype: int32

In [ ]:
# one hot encoding
cudf.get_dummies(df['cut'])

,Fair,Good,Ideal,Premium,Very Good
0,0,0,1,0,0
1,0,0,0,1,0
2,0,1,0,0,0
3,0,0,0,1,0
4,0,1,0,0,0
...,...,...,...,...,...
53935,0,0,1,0,0
53936,0,1,0,0,0
53937,0,0,0,0,1
53938,0,0,0,1,0


In [ ]:
num_labels = X_train.select_dtypes("number").columns
cat_labels = X_train.select_dtypes("category").columns


In [ ]:
cat_preprocessor = OneHotEncoder()
num_preprocessor = FunctionTransformer()

In [ ]:
cat_labels

Index(['cut', 'color', 'clarity'], dtype='object')

In [ ]:
preprocessor = ColumnTransformer([
    ("cat",cat_preprocessor, cat_labels),
    ("num",num_preprocessor, num_labels)

])

In [ ]:
preprocessor

ColumnTransformer()

In [ ]:
X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep = preprocessor.fit_transform(X_test)#tutaj powinien byc tylko transform!


In [ ]:
feature_labels = preprocessor.transformers_[0][1]\
.get_feature_names(cat_labels).tolist()
feature_labels.extend(num_labels)

In [ ]:
X_train_prep.columns = feature_labels
X_test_prep.columns = feature_labels


In [ ]:
X_train_prep

,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,...,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,carat,depth,table,x,y,z
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.61,62.8,59.0,5.44,5.30,3.37
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.24,61.7,55.0,6.91,6.96,4.28
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.19,59.9,61.0,6.91,6.85,4.12
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.54,58.9,59.0,5.37,5.32,3.15
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.01,62.7,58.0,8.02,7.97,5.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37753,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.80,62.4,57.0,5.90,5.87,3.67
37754,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.53,61.7,55.0,5.20,5.24,3.22
37755,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.20,61.0,56.0,6.86,6.88,4.19
37756,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.38,62.4,64.0,4.61,4.59,2.87


In [ ]:
X.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [ ]:
X_train_prep.columns

Index(['cut_Fair', 'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good',
       'color_D', 'color_E', 'color_F', 'color_G', 'color_H', 'color_I',
       'color_J', 'clarity_I1', 'clarity_IF', 'clarity_SI1', 'clarity_SI2',
       'clarity_VS1', 'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2', 'carat',
       'depth', 'table', 'x', 'y', 'z'],
      dtype='object')

# Xgboost

We are ready to build the XGBoost model. Let's go over the popular parameter settings the XGBoost first.

## General Parameters

These define the overall functionality of XGBoost.

1. booster [default=gbtree]
    - Select the type of model to run at each iteration. It has 2 options:
        - gbtree: tree-based models
        - gblinear: linear models
2. silent [default=0]:
    - Silent mode is activated is set to 1, i.e. no running messages will be printed.
    - It’s generally good to keep it 0 as the messages might help in understanding the model.
 

## Booster Parameters
Though there are 2 types of boosters, I’ll consider only tree booster here because it always outperforms the linear booster and thus the later is rarely used.

1. eta [default=0.3]
    - Makes the model more robust by shrinking the weights on each step
    - Typical final values to be used: 0.01-0.2
2. min_child_weight [default=1]
    - Defines the minimum sum of weights of all observations required in a child.
3. max_depth [default=6]
    - Used to control over-fitting as higher depth will allow model to learn relations very specific to a particular sample.
    - Should be tuned using CV.
    - Typical values: 3-10
4. max_leaf_nodes
    - The maximum number of terminal nodes or leaves in a tree.
    - Can be defined in place of max_depth. Since binary trees are created, a depth of ‘n’ would produce a maximum of 2^n leaves.
5. gamma [default=0]
    - A node is split only when the resulting split gives a positive reduction in the loss function. Gamma specifies the minimum loss reduction required to make a split.
    - Makes the algorithm conservative. The values can vary depending on the loss function and should be tuned.
6. subsample [default=1]
    - Denotes the fraction of observations to be randomly samples for each tree.
    - Lower values make the algorithm more conservative and prevents overfitting but too small values might lead to under-fitting.
    - Typical values: 0.5-1
7. colsample_bytree [default=1]
    - Denotes the fraction of columns to be randomly samples for each tree.
    - Typical values: 0.5-1
8. colsample_bylevel [default=1]
    - Denotes the subsample ratio of columns for each split, in each level.
    - I don’t use this often because subsample and colsample_bytree will do the job for you. but you can explore further if you feel so.
9. lambda [default=1]
    - L2 regularization term on weights (analogous to Ridge regression)
    - This used to handle the regularization part of XGBoost. Though many data scientists don’t use it often, it should be explored to reduce overfitting.
10. alpha [default=0]
    - L1 regularization term on weight (analogous to Lasso regression)
    - Can be used in case of very high dimensionality so that the algorithm runs faster when implemented
11. scale_pos_weight [default=1]
    - A value greater than 0 should be used in case of high class imbalance as it helps in faster convergence.
 

## Learning Task Parameters
These parameters are used to define the optimization objective the metric to be calculated at each step.

1. objective [default=reg:linear]
    - This defines the loss function to be minimized. Mostly used values are:
        - binary:logistic –logistic regression for binary classification, returns predicted probability (not class)
        - multi:softmax –multiclass classification using the softmax objective, returns predicted class (not probabilities)
            you also need to set an additional num_class (number of classes) parameter defining the number of unique classes
        - multi:softprob –same as softmax, but returns predicted probability of each data point belonging to each class.
2. eval_metric [ default according to objective ]
    - The metric to be used for validation data.
    - The default values are rmse for regression and error for classification.
    - Typical values are:
        - rmse – root mean square error
        - mae – mean absolute error
        - logloss – negative log-likelihood
        - error – Binary classification error rate (0.5 threshold)
        - merror – Multiclass classification error rate
        - mlogloss – Multiclass logloss
        - auc: Area under the curve
3. seed [default=0]
    - The random number seed.
    - Can be used for generating reproducible results and also for parameter tuning.
    
Let's start with a set of parameter to train the model. We will come back to tune the parameters later.

## Training XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
?xgb.DMatrix #format for xgb

In [ ]:
val_frac = .2
val_split = int(X_train_prep.shape[0]*(1-val_frac))

In [ ]:
val_split

30206

In [ ]:
X_train_prep.index

RangeIndex(start=0, stop=37758, step=1)

In [ ]:
dtrain = xgb.DMatrix(X_train_prep.iloc[:val_split], y_train[:val_split])
dval = xgb.DMatrix(X_train_prep.iloc[val_split:], y_train[val_split:])
dtest = xgb.DMatrix(X_test_prep, y_test)


In [ ]:
params = {
    'objective': 'reg:squarederror',
    'tree_method': "gpu_hist",
    'eval_metric': 'rmse'
}

trained_model = xgb.train(params,dtrain, 
                          num_boost_round=500,
                         evals = [(dtrain,'train'),
                                 (dval, "val")])

[0]	train-rmse:4003.86650	val-rmse:4005.01887
[1]	train-rmse:2891.44040	val-rmse:2886.43908
[2]	train-rmse:2127.93177	val-rmse:2123.25831
[3]	train-rmse:1610.80969	val-rmse:1605.25515
[4]	train-rmse:1268.31853	val-rmse:1270.25888
[5]	train-rmse:1033.05207	val-rmse:1041.29982
[6]	train-rmse:883.96418	val-rmse:899.95327
[7]	train-rmse:790.73341	val-rmse:814.42358
[8]	train-rmse:722.14391	val-rmse:751.25210
[9]	train-rmse:678.17122	val-rmse:716.25917
[10]	train-rmse:647.22049	val-rmse:688.03040
[11]	train-rmse:619.40613	val-rmse:666.72873
[12]	train-rmse:603.87973	val-rmse:655.63534
[13]	train-rmse:589.51574	val-rmse:644.93790
[14]	train-rmse:577.96548	val-rmse:635.67213
[15]	train-rmse:566.73752	val-rmse:628.00511
[16]	train-rmse:557.98720	val-rmse:623.10204
[17]	train-rmse:550.43904	val-rmse:619.39353
[18]	train-rmse:543.62535	val-rmse:618.13450
[19]	train-rmse:534.27995	val-rmse:611.72611
[20]	train-rmse:530.22253	val-rmse:609.47797
[21]	train-rmse:523.42376	val-rmse:606.44929
[22]	tra

[181]	train-rmse:295.24228	val-rmse:583.26322
[182]	train-rmse:294.40141	val-rmse:583.47743
[183]	train-rmse:293.83802	val-rmse:583.56575
[184]	train-rmse:293.45117	val-rmse:583.42134
[185]	train-rmse:292.13740	val-rmse:583.69804
[186]	train-rmse:291.87483	val-rmse:583.65845
[187]	train-rmse:291.79828	val-rmse:583.69887
[188]	train-rmse:291.75698	val-rmse:583.68290
[189]	train-rmse:291.38081	val-rmse:583.80672
[190]	train-rmse:290.62096	val-rmse:583.76177
[191]	train-rmse:290.16260	val-rmse:583.70793
[192]	train-rmse:289.83706	val-rmse:583.79954
[193]	train-rmse:288.48460	val-rmse:584.23931
[194]	train-rmse:287.40286	val-rmse:584.38945
[195]	train-rmse:286.62160	val-rmse:584.41241
[196]	train-rmse:286.26242	val-rmse:584.43106
[197]	train-rmse:285.04275	val-rmse:584.57283
[198]	train-rmse:284.37170	val-rmse:584.60203
[199]	train-rmse:284.15297	val-rmse:584.61671
[200]	train-rmse:283.71524	val-rmse:584.66806
[201]	train-rmse:283.27620	val-rmse:584.52709
[202]	train-rmse:282.52624	val-rms

[360]	train-rmse:214.52119	val-rmse:591.53672
[361]	train-rmse:213.81529	val-rmse:591.42752
[362]	train-rmse:213.43485	val-rmse:591.41823
[363]	train-rmse:213.28936	val-rmse:591.38895
[364]	train-rmse:213.24069	val-rmse:591.37374
[365]	train-rmse:212.76525	val-rmse:591.35100
[366]	train-rmse:212.22026	val-rmse:591.56570
[367]	train-rmse:211.79252	val-rmse:591.47027
[368]	train-rmse:211.58094	val-rmse:591.51204
[369]	train-rmse:211.34469	val-rmse:591.48229
[370]	train-rmse:210.94984	val-rmse:591.65272
[371]	train-rmse:210.62581	val-rmse:591.69052
[372]	train-rmse:210.11797	val-rmse:591.62394
[373]	train-rmse:209.78541	val-rmse:591.84146
[374]	train-rmse:209.64986	val-rmse:591.80626
[375]	train-rmse:209.64264	val-rmse:591.82536
[376]	train-rmse:209.62017	val-rmse:591.82290
[377]	train-rmse:209.05492	val-rmse:591.90942
[378]	train-rmse:208.48246	val-rmse:592.11134
[379]	train-rmse:208.40471	val-rmse:592.08720
[380]	train-rmse:208.18536	val-rmse:591.99552
[381]	train-rmse:207.91489	val-rms

In [ ]:
y_test_pred = trained_model.predict(dtest)
y_test_pred


numpy.ndarray

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test.to_numpy(), y_test_pred, squared = False)

571.8253474596264


Try to adjust the `num_boost_round` to see the effect in training

num_boost_round 250
learning_rate .05 .1 .2

In [ ]:
params = {
    'learning_rate': .1,
    'objective': 'reg:squarederror',
    'tree_method': "gpu_hist",
    'eval_metric': 'rmse'
}

In [ ]:
verbose

In [ ]:
%%timeit
trained_model = xgb.train(params,dtrain, 
                          num_boost_round=500)#,
                         #evals = [(dtrain,'train'),
                          #       (dval, "val")])

1.4 s ± 8.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
y_test_pred = trained_model.predict(dtest)
y_test_pred


array([ 786.93365, 1571.508  ,  634.70416, ..., 4377.552  ,  405.1752 ,
        836.00696], dtype=float32)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test.to_numpy(), y_test_pred, squared = False)

551.3741411385047

## XGBoost in Sklearn API
XGBoost also provides a Scikit-learn wrapper interface for XGBoost model, which makes it easy to use the tools inside Scikit-learn library. Let's train it with Scikit-learning API. 

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor(**params,n_estimators = 500)
model.fit(X_train_prep.iloc[:val_split], y_train[:val_split])

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmse', gamma=0, gpu_id=0, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [ ]:
%%timeit
model.fit(X_train_prep.iloc[:val_split], y_train[:val_split])

1.42 s ± 9.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [1]:
params

NameError: ignored

In [ ]:
y_test_pred = model.predict(X_test_prep)
mean_squared_error(y_test.to_numpy(), y_test_pred, squared = False)

551.3741411385047

## Improving the performance of our models - Hyperparameters tuning

One of the most useful components beyond basic models that Scikit-learn offers is hyperparameter optimization for its models. Hyperparameter optimization means looking for the values of parameters that maximize how well our model can predict observations in `X_test`. 

Fortunately cuML is compatible with Scikit-learn hyperparameter optimization!!! Note: It also is compatible with other libraries, such as dask-ml that perform hyperparameter optimization with more advanced strategies/levels of parallelization.

For this lets use the `GridSearchCV` class of Scikit-learn, which like the name suggest performs a search in a grid of values of the parameters that we specify:

In [ ]:
from sklearn.tree import DecisionTreeRegressor, export_graphviz

In [ ]:
sk_tree = DecisionTreeRegressor(max_depth = 5,random_state=42)

sk_tree.fit(X_train_prep.to_pandas(),y_train.to_pandas())

export_graphviz(sk_tree, "tree.dot",feature_names=feature_labels)

In [ ]:
!dot -Tpng tree.dot -o tree.png

In [ ]:
X_train_prep.shape

(37758, 26)

In [ ]:
y_train.mean()

3933.1851528152974

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold

model = XGBRegressor()

hyper_params = {
    'learning_rate': [.1],
    'colsample_bytree': [.6,.8,1],
    'colsample_bylevel': [1],#[.8,1],
    'alpha': [0,5],
    'max_depth': [6,7,8],
    'n_estimators': [200,500],
    'objective': ['reg:squarederror'],
    'tree_method': ["gpu_hist"],
    'eval_metric': ['rmse']
}

kfold = KFold(n_splits=5,shuffle=True,random_state=42)

grid_search = GridSearchCV(model,hyper_params,
                           scoring = "neg_root_mean_squared_error",
                          cv = kfold,
                           verbose = 1)

In [ ]:
X_train_prep_numpy = X_train_prep.values.get()
y_train_numpy = y_train.values.get()

In [ ]:
grid_search.fit(X_train_prep_numpy, y_train_numpy)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints...
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None, ...),
             param_grid={'alpha': [0, 5], 'colsample_bylevel': [1],
                         'colsample_bytree': [0.6, 0.8, 1],
                         'eval_metric': ['rmse'], 'learning_rate': [0.1],
                         'max_depth': [6, 7, 8], 'n_estimators': [200, 500],
                         'objective': ['reg:squarederror'],
                         'tree_method': ['gpu_hist']},
             scoring='neg_root_mean_squared_error', verbose=1)

In [ ]:
grid_search.best_params_

{'alpha': 5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'eval_metric': 'rmse',
 'learning_rate': 0.1,
 'max_depth': 7,
 'n_estimators': 200,
 'objective': 'reg:squarederror',
 'tree_method': 'gpu_hist'}

In [ ]:
y_test_pred = grid_search.predict(X_test_prep)

In [ ]:
mean_squared_error(y_test.to_numpy(),y_test_pred,squared=False)

533.9617423540477

## Forest Inference Library

The open source Forest Inference Library (FIL) in RAPIDS can accelerate GBDT and RF inference with GPUs. Users can train models as usual in XGBoost or LightGBM, save them to disk, then use FIL to reload those models and apply them to new data. Using FIL, a single V100 GPU can deliver up to 35x more inference throughput than a CPU-only node with 40 cores. For more details, please refer to this [blog](https://medium.com/rapids-ai/rapids-forest-inference-library-prediction-at-100-million-rows-per-second-19558890bc35).

The XGBoost model can be serialized to disk by `save_model` command:

In [ ]:
grid_search.best_estimator_.save_model('best.model')

In [ ]:
from cuml import ForestInference
fm = ForestInference.load('best.model',output_class=False)
preds_gpu = fm.predict(X_test_prep)
preds_gpu

0          774.125977
1         1605.511108
2          666.813721
3        13314.742188
4         3484.948975
             ...     
16177     4851.317871
16178      690.293152
16179     4227.971191
16180      423.533600
16181      809.185974
Length: 16182, dtype: float32

In [ ]:
from cuml.metrics import mean_squared_error

In [ ]:
%%timeit
mean_squared_error(y_test,preds_gpu,squared=False)

651 µs ± 7.82 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
y_test_cpu = y_test.to_numpy()

In [ ]:
%%timeit
mean_squared_error(y_test_cpu,y_test_pred,squared=False)

800 µs ± 13.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
preds_gpu = fm.predict(X_test_prep)


8.35 ms ± 195 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
y_test_pred = grid_search.predict(X_test_prep)

8.34 ms ± 151 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
